# 2.2 Wrangling

In this notebook, we practice with operations on tidy data frames.

In [149]:
# imports

import pandas as pd
import json, os, codecs
from collections import defaultdict, OrderedDict
import seaborn as sns

## Import the datasets

Let us import sample dataset in memory. They contain different metadata about newspaper titles. One of them contains metadata at the level of articles. A second and third contain different metadata at the higher level of titles. This will be a typical case of data about the same objects scattered across different dfs. 

In [318]:
root_folder = "../data/lwmnewspapers/"
df_articles = pd.read_csv(os.path.join(root_folder,"LwM-HMD-articles.csv"))
df_MPD_links = pd.read_csv(os.path.join(root_folder,"MPD_links.csv"))
df_wikiid_latlong = pd.read_csv(os.path.join(root_folder,"wikiid_lat_long.csv"))

## Take a look
Let's take another look to the dataset to refresh out memories.

In [319]:
df_articles.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
0,3077,126,art0037,"Nelson Chronicle, Colne Observer, and Clithero...",British Library Living with Machines Project,1894-01-26,1894,1,26,"Nelson, Lancashire, England",174,0.8530,1890
1,3040,326,art0056,The Birkenhead News and Wirral General Adverti...,British Library Living with Machines Project,1887-03-26,1887,3,26,"Birkenhead, Merseyside, England",509,0.9844,1880


In [152]:
df_MPD_links.head(2)

,NLP,year,wikidata_id,S-PRICE
0,51,1846,Q84,5d
1,1260,1846,Q84,3 ½


In [153]:
df_wikiid_latlong.head(5)

,wikidata_id,latitude,longitude
0,Q84,51.507222,-0.1275
1,Q84,51.507222,-0.1275
2,Q84,51.507222,-0.1275
3,Q84,51.507222,-0.1275
4,Q84,51.507222,-0.1275


In [320]:
df_articles.shape

(100, 13)

In [321]:
df_MPD_links.shape

(9124, 4)

In [156]:
df_wikiid_latlong.shape

(9124, 3)

In [262]:
# let's create a few smaller datasets to play with transformations, via selection

df_articles_93 = df_articles[df_articles["year"] == 1893]
df_articles_87 = df_articles[df_articles["year"] == 1887]

In [263]:
df_articles_93.shape

(148198, 13)

In [264]:
df_articles_87.shape

(68438, 13)

In [265]:
df_articles_93.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
0,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890
1,2596,1215,art0005,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",7,0.5200,1890


In [266]:
df_articles_87.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
8900,2596,819,art0030,The Cotton Factory Times.,British Library Living with Machines Project,1887-08-19,1887,8,19,"Manchester, Greater Manchester, England",10,0.8460,1880
8901,2596,819,art0097,The Cotton Factory Times.,British Library Living with Machines Project,1887-08-19,1887,8,19,"Manchester, Greater Manchester, England",13,0.8315,1880


Note that the index of the original df is maintained, unless you specify to reset the index at the time of subsetting or merging dataframes (not specifying `drop=True` would keep the original index as a new column `index`):

In [292]:
df_articles_93 = df_articles[df_articles["year"] == 1893].reset_index(drop=True)
df_articles_87 = df_articles[df_articles["year"] == 1887].reset_index(drop=True)
df_articles_87.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
0,2596,819,art0030,The Cotton Factory Times.,British Library Living with Machines Project,1887-08-19,1887,8,19,"Manchester, Greater Manchester, England",10,0.8460,1880
1,2596,819,art0097,The Cotton Factory Times.,British Library Living with Machines Project,1887-08-19,1887,8,19,"Manchester, Greater Manchester, England",13,0.8315,1880


## Set operations

In [200]:
df_articles_87['location'].unique()

array(['Manchester, Greater Manchester, England',
       'Widnes, Cheshire, England', 'Denton, Greater Manchester, England',
       'Stourbridge, West Midlands, England',
       'Stockton-on-Tees, Cleveland, England',
       'Weymouth, Dorset, England', 'Swansea, West Glamorgan, Wales',
       'Birkenhead, Merseyside, England',
       'Atherstone, Warwickshire, England',
       'Birmingham, West Midlands, England',
       'Bridgend, Mid Glamorgan, Wales',
       'Bridlington, Humberside, England',
       'Brighouse, West Yorkshire, England'], dtype=object)

or:

In [201]:
set(df_articles_87['location'])

{'Atherstone, Warwickshire, England',
 'Birkenhead, Merseyside, England',
 'Birmingham, West Midlands, England',
 'Bridgend, Mid Glamorgan, Wales',
 'Bridlington, Humberside, England',
 'Brighouse, West Yorkshire, England',
 'Denton, Greater Manchester, England',
 'Manchester, Greater Manchester, England',
 'Stockton-on-Tees, Cleveland, England',
 'Stourbridge, West Midlands, England',
 'Swansea, West Glamorgan, Wales',
 'Weymouth, Dorset, England',
 'Widnes, Cheshire, England'}

As a Python refresher on sets vs lists:

In [205]:
A = set(['aa', 'abc', 'bb', 'aa'])
A

{'aa', 'abc', 'bb'}

In [206]:
A = set([1,2,3,4,4,5])
B = set([1,2,3,4,4,5,7,8,9])

In [207]:
A

{1, 2, 3, 4, 5}

In [208]:
B

{1, 2, 3, 4, 5, 7, 8, 9}

In [209]:
print(A.union(B))

{1, 2, 3, 4, 5, 7, 8, 9}


In [210]:
print(A.difference(B))

set()


In [211]:
print(B.difference(A))

{8, 9, 7}


In [212]:
print(A.intersection(B))

{1, 2, 3, 4, 5}


### Projection

In [214]:
df_articles[["NLP","year"]].head(10)

,NLP,year
0,2596,1893
1,2596,1893
2,2596,1893
3,2596,1893
4,2596,1893
5,2596,1893
6,2596,1893
7,2596,1893
8,2596,1893
9,2596,1893


### Union

In [267]:
df_temp = pd.concat([df_articles_87, df_articles_93])

In [268]:
df_temp.shape

(216636, 13)

### Difference

In [285]:
df_articles_87['location'].isin(df_articles_93['location']).value_counts()

location
True     44447
False    23991
Name: count, dtype: int64

In [288]:
df_articles_87.shape

(68438, 13)

In [289]:
df_articles_87[df_articles_87['location'].isin(df_articles_93['location'])].shape

(44447, 13)

In [290]:
df_articles_87[~df_articles_87['location'].isin(df_articles_93['location'])].shape

(23991, 13)

In [291]:
df_articles_87[~df_articles_87['location'].isin(df_articles_87['location'])].shape

(0, 13)

### Intersection

In [188]:
pd.merge(right=df_book_tragedy, left=df_book_tragedy, how="inner").shape

(32, 12)

In [189]:
pd.merge(right=df_book_tragedy, left=df_book_comedy, how="inner").shape

(0, 12)

## Joins

![figure with joins](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png?ezimgfmt=ng:webp/ngcb1)

- **Inner Join** or Natural join: To keep only rows that match from the data frames, specify the argument `how=‘inner’`.
- **Outer Join** or Full outer join:To keep all rows from both data frames, specify `how=‘outer’`.
- **Left Join** or Left outer join:To include all the rows of your data frame x and only those from y that match, specify `how=‘left’`.
- **Right Join** or Right outer join:To include all the rows of your data frame y and only those from x that match, specify `how=‘right’`.

In [293]:
df_articles.shape

(2019676, 13)

In [294]:
df_MPD_links.shape

(9124, 4)

In [295]:
df_wikiid_latlong.shape

(9124, 3)

In [298]:
df_articles.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
0,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890
1,2596,1215,art0005,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",7,0.5200,1890


In [297]:
df_MPD_links.head(2)

,NLP,year,wikidata_id,S-PRICE
0,51,1846,Q84,5d
1,1260,1846,Q84,3 ½


The two dfs have two columns in common: `NLP` and `year`. To add any column in `metadf` which is not in `articlesdf` to `articlesdf` based on a match of both `NLP` and `year`, you simply do a left-merge:

In [299]:
df_articles.head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade
0,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890
1,2596,1215,art0005,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",7,0.5200,1890


In [300]:
df_articles.merge(df_MPD_links,how='left').head(2)

,NLP,issue,art_num,title,collection,full_date,year,month,day,location,word_count,ocrquality,decade,wikidata_id,S-PRICE
0,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890,Q18125,1d
1,2596,1215,art0005,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",7,0.5200,1890,Q18125,1d


It automatically uses all common columns to merge the left df with the right. If we specify one column only, then the result is different:

In [301]:
df_articles.merge(df_MPD_links,how='left',on='NLP').shape

(18809982, 16)

This is because in `metadf` each `NLP`-`year` combination has a unique set of variables. If we remove `year` off the equation when merging, then pandas will consider `year` from each df as different variables and it will add all combinations of NLP-year from both datasets. Note that in that case, the `year` col will be automatically renamed as `year_x` (left df) and `year_y` (right df):

In [302]:
df_articles.merge(df_MPD_links,how='left',on='NLP').head(2)

,NLP,issue,art_num,title,collection,full_date,year_x,month,day,location,word_count,ocrquality,decade,year_y,wikidata_id,S-PRICE
0,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890,1886.0,Q18125,1d
1,2596,1215,art0089,The Cotton Factory Times.,British Library Living with Machines Project,1893-12-15,1893,12,15,"Manchester, Greater Manchester, England",3,0.5533,1890,1888.0,Q18125,1d


Inner merge will get you the intersection between the two dfs based on the common columns:

In [303]:
pd.merge(right=df_articles, left=df_MPD_links, how="inner").shape
# also equivalent:
# df_articles.merge(df_MPD_links,how="inner")

(1132863, 15)

Let's play around with the inner-merged df...

In [304]:
articlesdf_merged_inner = pd.merge(right=df_articles, left=df_MPD_links, how="inner")
articlesdf_merged_inner.head(2)

,NLP,year,wikidata_id,S-PRICE,issue,art_num,title,collection,full_date,month,day,location,word_count,ocrquality,decade
0,3040,1880,Q746718,1d,320,art0038,The Birkenhead News and Wirral General Adverti...,British Library Living with Machines Project,1880-03-20,3,20,"Birkenhead, Merseyside, England",3,0.3000,1880
1,3040,1880,Q746718,1d,320,art0025,The Birkenhead News and Wirral General Adverti...,British Library Living with Machines Project,1880-03-20,3,20,"Birkenhead, Merseyside, England",79,0.7615,1880


If we wanted to use a subset column-wise:

In [306]:
articlesdf_merged_inner = articlesdf_merged_inner[["NLP","year","location","wikidata_id","S-PRICE"]]
articlesdf_merged_inner.tail(2)

,NLP,year,location,wikidata_id,S-PRICE
1132861,2982,1900,"Weymouth, Dorset, England",Q550110,½ d<SEP>1d
1132862,2982,1900,"Weymouth, Dorset, England",Q550110,½ d<SEP>1d


Let's check which years are available:

In [307]:
articlesdf_merged_inner['year'].value_counts()

year
1893    141805
1891     98059
1883     95832
1895     86480
1889     83963
1890     83641
1888     79652
1885     75535
1898     74464
1881     74281
1886     72286
1880     71495
1900     55249
1896     40121
Name: count, dtype: int64

The counts itself could be turned into a df if useful:

In [308]:
yearcounts = pd.DataFrame(articlesdf_merged_inner['year'].value_counts()).reset_index()
yearcounts

,year,count
0,1893,141805
1,1891,98059
2,1883,95832
3,1895,86480
4,1889,83963
5,1890,83641
6,1888,79652
7,1885,75535
8,1898,74464
9,1881,74281


... and exported as csv if needed:

In [125]:
# yearcounts.to_csv(os.path.join(root_folder,"LwM-HMD-articles.csv",index=False))

We may want to analyse articles from a specific year:

In [309]:
articlesdf_merged_inner[articlesdf_merged_inner['year'] == 1885].head()

,NLP,year,location,wikidata_id,S-PRICE
241608,3045,1885,"Atherstone, Warwickshire, England",Q747778,1d
241609,3045,1885,"Atherstone, Warwickshire, England",Q747778,1d
241610,3045,1885,"Atherstone, Warwickshire, England",Q747778,1d
241611,3045,1885,"Atherstone, Warwickshire, England",Q747778,1d
241612,3045,1885,"Atherstone, Warwickshire, England",Q747778,1d


... or in all but one year:

In [310]:
articlesdf_merged_inner[articlesdf_merged_inner['year'] != 1885].head()

,NLP,year,location,wikidata_id,S-PRICE
0,3040,1880,"Birkenhead, Merseyside, England",Q746718,1d
1,3040,1880,"Birkenhead, Merseyside, England",Q746718,1d
2,3040,1880,"Birkenhead, Merseyside, England",Q746718,1d
3,3040,1880,"Birkenhead, Merseyside, England",Q746718,1d
4,3040,1880,"Birkenhead, Merseyside, England",Q746718,1d


You may want to make sure that the df has no NaN in one specific column or across the df. Check if that's the case:

In [311]:
print(articlesdf_merged_inner.shape)
print(articlesdf_merged_inner[~articlesdf_merged_inner["location"].isna()].shape)

(1132863, 5)
(1125286, 5)


As you can see, the df without rows in which the column `location` is NaN has fewer rows. You can remove the rows if needed:

In [312]:
articlesdf_merged_inner = articlesdf_merged_inner[~articlesdf_merged_inner["location"].isna()]

or equally:

In [313]:
articlesdf_merged_inner.dropna(subset=['location'],inplace=True)

Without adding a `subset` it would remove all rows in which at least one column is NaN. In highly sparse datasets, you may end up with no rows... so careful.

**⏰ ✏️ Excercise**:

* Using the most suitable type of merging, create a df starting from `df_articles` and adding any column not present there from `df_MPD_links` and `df_wikiid_latlong`. Make sure that the final dataframe has the same number of rows as `df_articles`!
* Which is the most represented `location` in the dataframe?

In [ ]:
### Write your solution here

## Pivoting

This is bonus content!

For more (including stacking with multi-indexes and unpivoting or melting), see https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [314]:
data = {'place': ["London","London","Berlin","Berlin","Rome","Rome"],
       'year': [1800, 1900, 1800, 1900, 1800, 1900],
       'values': [10,20,30,40,50,60]}
toy_df = pd.DataFrame(data, columns=['place','year','values'])

In [315]:
toy_df

,place,year,values
0,London,1800,10
1,London,1900,20
2,Berlin,1800,30
3,Berlin,1900,40
4,Rome,1800,50
5,Rome,1900,60


In [316]:
pivoted = toy_df.pivot(index='year', columns='place', values='values')

In [317]:
pivoted

place,Berlin,London,Rome
year,,,
1800,30,10,50
1900,40,20,60
